In [1]:
import pandas as pd
import numpy as np
import sys, os
import json

import matplotlib.pyplot as plt
import seaborn as sns

greek_cols = [
	'delta',
	'gamma',
	'theta',
	'vega',
	'rho',
	'vanna',
	'vomma',
	'charm',
	'veta',
	'speed',
	'zomma',
	'color',
	'ultima'
]

### Data

In [2]:
options = pd.read_csv("data/filtered_tsla_options_w_greeks.csv")
options = options[options.date_current == "2020-07-13"]
options = options[options.days_to_expiry.isin([95.0, 186.0, 249.0])]

strikes = options.strike_price.value_counts()
strikes = strikes[strikes >= 3].index
options = options[options.strike_price.isin(strikes)].reset_index(drop=True)

### Results

In [3]:
with open("cplex_results/results_18.json", "r") as file:
    results = json.loads(json.loads(file.read()))

In [4]:
legs = results['CPLEXSolution']['variables']

directions = [
    float(leg['value']) if leg['name'][0] == 'l' else float(leg['value']) * -1
    for leg in legs
]
idc = [
    int(leg['name'][1:])
    for leg in legs
]

print("Indices", idc)
print("Directions", directions)

Indices [131, 284, 96, 255]
Directions [2.0, 2.0, -2.0, -2.0]


### Collect Legs

In [5]:
position_legs = options.iloc[idc].reset_index(drop=True)
position_legs['Direction'] = directions

### Check Position

In [8]:
position_map = position_legs[['option_id', 'Direction']]
position_map

,option_id,Direction
0,TSLA 2021-03-19 C232,2.0
1,TSLA 2021-01-15 P362,2.0
2,TSLA 2021-01-15 C300,-2.0
3,TSLA 2021-01-15 P302,-2.0


### Analyze Greeks

In [6]:
greeks = position_legs.loc[:, greek_cols]
greeks = pd.concat([
    greeks.multiply(position_legs.Direction, axis=0),
    greeks.multiply(position_legs.Direction, axis=0).sum(axis=0).to_frame().T
]).T
cols = position_legs.option_id.values.tolist()
cols = [f"{direction} x {col}" for direction, col in zip(position_legs.Direction, cols)]
greeks.columns = cols + ["Net"]

In [7]:
greeks

,2.0 x TSLA 2021-03-19 C232,2.0 x TSLA 2021-01-15 P362,-2.0 x TSLA 2021-01-15 C300,-2.0 x TSLA 2021-01-15 P302,Net
delta,1.525312,-1.002304,-1.263190,0.742812,0.002630
gamma,0.002772,0.004346,-0.003714,-0.003706,-0.000302
theta,-0.276564,-0.393814,0.411474,0.414708,0.155804
vega,1.528228,1.705354,-1.611720,-1.615980,0.005882
rho,1.545456,-2.697338,-1.119738,1.964006,-0.307614
vanna,0.000202,0.005722,-0.002714,-0.002798,0.000412
vomma,-0.035728,0.003532,0.194986,0.196276,0.359066
charm,-0.000036,-0.001322,0.000694,0.000718,0.000054
veta,0.003004,0.004592,-0.003834,-0.003840,-0.000078
speed,-0.000018,-0.000014,0.000018,0.000018,0.000004


### Analyze Profit

In [9]:
optionsl = pd.read_csv("data/filtered_tsla_options_w_greeks.csv")
optionsl =  optionsl[optionsl.date_current.isin(["2020-07-13", "2020-07-27"])]
optionsl = optionsl[optionsl.option_id.isin(position_legs.option_id)]
optionsl = optionsl.merge(position_map, on=['option_id'])
optionsl['mid'] = (optionsl.ask_price + optionsl.bid_price) * 0.5
optionsl['cost'] = optionsl.mid * optionsl.Direction
optionsl.groupby('date_current').apply(lambda x: x.cost.sum()).diff()

date_current
2020-07-13      NaN
2020-07-27    26.96
dtype: float64